In [75]:
import Pkg;
Pkg.add("PyPlot")
using PyPlot
using Random, LinearAlgebra, Statistics, DataFrames, Plots
Pkg.add("CSV")
using CSV

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [76]:
using LowRankModels

In [106]:
data = CSV.read("weather.csv")
head(data)

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f
,String,Int64?,Int64?,Int64?,Int64?
1,8/29/13,74,68,61,61
2,8/30/13,78,69,60,61
3,8/31/13,71,64,57,57
4,9/1/13,74,66,58,60
5,9/2/13,75,69,62,61
6,9/3/13,73,67,60,59


In [108]:
# Look at the column names
feature_names = names(data)
for i in 1:size(data)[2]
    println(string(i), "\t", string(feature_names[i]), "\t", string(eltype(data[!, i])))
end

1	date	String
2	max_temperature_f	Union{Missing, Int64}
3	mean_temperature_f	Union{Missing, Int64}
4	min_temperature_f	Union{Missing, Int64}
5	max_dew_point_f	Union{Missing, Int64}
6	mean_dew_point_f	Union{Missing, Int64}
7	min_dew_point_f	Union{Missing, Int64}
8	max_humidity	Union{Missing, Int64}
9	mean_humidity	Union{Missing, Int64}
10	min_humidity	Union{Missing, Int64}
11	max_sea_level_pressure_inches	Union{Missing, Float64}
12	mean_sea_level_pressure_inches	Union{Missing, Float64}
13	min_sea_level_pressure_inches	Union{Missing, Float64}
14	max_visibility_miles	Union{Missing, Int64}
15	mean_visibility_miles	Union{Missing, Int64}
16	min_visibility_miles	Union{Missing, Int64}
17	max_wind_Speed_mph	Union{Missing, Int64}
18	mean_wind_speed_mph	Union{Missing, Int64}
19	max_gust_speed_mph	Union{Missing, Int64}
20	precipitation_inches	Union{Missing, Float64}
21	cloud_cover	Union{Missing, Int64}
22	events	Union{Missing, String}
23	wind_dir_degrees	Union{Missing, Int64}
24	zip_code	Int64


In [109]:
# inserts "None" for every blank cell in Events 
# data.events = coalesce.(data.events, "None")

# TODO: delete row 2912 (since that row is completely empty: 8/8/15)

In [110]:
# locate missing data in the precipitation_inches column (column #20)
missingList = []

for row in 1:size(data)[1]
    if ismissing(data[row,20])
        push!(missingList, (20,row))
    end
end

# in total there are 143 missing elements that come from:
# cols 20
# rows 33, 60, 76 ... , 3653

missingList

144-element Array{Any,1}:
 (20, 33)
 (20, 60)
 (20, 76)
 (20, 85)
 (20, 96)
 (20, 132)
 (20, 134)
 (20, 136)
 (20, 154)
 (20, 155)
 (20, 166)
 (20, 171)
 (20, 373)
 ⋮
 (20, 3525)
 (20, 3536)
 (20, 3561)
 (20, 3564)
 (20, 3582)
 (20, 3604)
 (20, 3606)
 (20, 3612)
 (20, 3613)
 (20, 3622)
 (20, 3634)
 (20, 3653)

In [111]:
# impute missing data
# losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]
# rw = Regularizer[QuadReg(), QuadReg(), OneReg()]

data = data[:,2:21]

,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f
,Int64?,Int64?,Int64?,Int64?,Int64?
1,74,68,61,61,58
2,78,69,60,61,58
3,71,64,57,57,56
4,74,66,58,60,56
5,75,69,62,61,60
6,73,67,60,59,56
7,74,68,61,59,57
8,72,66,60,57,56
9,85,71,56,57,51


In [112]:
# impute missing data
# losses = Loss[QuadLoss(), LogisticLoss(), HingeLoss()]
# rw = Regularizer[QuadReg(), QuadReg(), OneReg()]

Y = data
l2 = QuadLoss()
l1 = L1Loss()
nonneg = NonNegConstraint()
quad = QuadReg()
lin = OneReg()
zero = ZeroReg()
k = size(data)[2]

Ω = [(x,y) for x=1:size(data)[1] , y=1:size(data)[2] if !ismissing(data[x,y])];

In [113]:
glrm = GLRM(Y, l1, quad, quad, k, obs=Ω)
X,W,ch = fit!(glrm, maxiter=1000)

Fitting GLRM
Iteration 10: objective value = 235957.15047449208
Iteration 20: objective value = 154649.80341078053
Iteration 30: objective value = 118212.17888411047
Iteration 40: objective value = 104296.74068819407
Iteration 50: objective value = 96990.41016436975
Iteration 60: objective value = 92962.11839368596
Iteration 70: objective value = 90342.75162101831
Iteration 80: objective value = 88469.59460815018
Iteration 90: objective value = 87165.7164524791
Iteration 100: objective value = 86064.97690892006


([0.3897166832568605 0.8687070737616609 … 0.7517461045546836 1.1510990806529349; -0.22463188656062602 -0.8841589085208649 … -0.9534371433772229 -0.30447722300758245; … ; 1.2036346525581214 0.8315647027612765 … 0.7459642258133946 1.4414461315196136; -0.2146618619866524 0.3689563034677936 … 1.332604174832134 1.2981371692813681], [5.39456801172819 4.8862507202872125 … 0.9185592614646534 -0.8077788007280942; -0.7035091121420111 -1.0732613349364652 … -1.1190590695750573 -0.08948186627411146; … ; 5.492789216361133 4.284830568784399 … 0.5287859442371065 0.2903284992230952; 4.824648753926481 4.061347709204127 … 0.7404450836334228 -0.014011658797938647], ConvergenceHistory("ProxGradGLRM", [2.6084445799052287e6, 2.485459231691199e6, 2.2969431272329143e6, 1.484104821690426e6, 633039.0283837395, 433684.9090956645, 353650.86458601896, 300869.88251595537, 272206.15005448606, 249896.32992373902  …  87050.58888653036, 86931.50690669953, 86797.94320644831, 86672.6873539754, 86552.92618409861, 86443.363

In [114]:
imputed_data = impute_missing(glrm)

3665×20 Array{Number,2}:
 74  68  61  61  58  56  93  75  57  30.07  …  10  10  10  23  11  28  0.0  4
 78  69  60  61  58  56  90  70  50  30.05     10  10   7  29  13  35  0.0  2
 71  64  57  57  56  54  93  75  57  30.0      10  10  10  26  15  31  0.0  4
 74  66  58  60  56  53  87  68  49  29.96     10  10  10  25  13  29  0.0  4
 75  69  62  61  60  58  93  77  61  29.97     10  10   6  23  12  30  0.0  6
 73  67  60  59  56  51  84  65  46  30.02  …  10  10  10  24  15  31  0.0  2
 74  68  61  59  57  56  90  72  53  30.05     10  10  10  29  19  35  0.0  4
 72  66  60  57  56  54  90  74  57  30.04     10  10  10  31  21  37  0.0  3
 85  71  56  57  51  45  86  58  29  30.0      10  10  10  24   8  28  0.0  0
 88  73  58  64  54  46  86  59  31  29.88     10  10  10  21   8  25  0.0  1
 74  65  56  58  54  52  86  70  53  29.88  …  10  10  10  26  11  31  0.0  2
 76  66  55  58  55  52  90  70  50  29.82     10  10  10  26  11  30  0.0  1
 74  66  57  59  56  54  93  73  53  29

In [115]:
convert(DataFrame, imputed_data)

# ensures that precipitation_inches >= 0
for i = 1:3665
    imputed_data[i,19] = max(0, imputed_data[i,19]);
end

imputed_data[31:35,19]

5-element Array{Number,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [116]:
final_imputed_data = CSV.read("weather.csv")
final_imputed_data[:, 20] = imputed_data[:, 19] # precipiaion_inches
final_imputed_data

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f
,String,Int64?,Int64?,Int64?,Int64?
1,8/29/13,74,68,61,61
2,8/30/13,78,69,60,61
3,8/31/13,71,64,57,57
4,9/1/13,74,66,58,60
5,9/2/13,75,69,62,61
6,9/3/13,73,67,60,59
7,9/4/13,74,68,61,59
8,9/5/13,72,66,60,57
9,9/6/13,85,71,56,57


In [118]:
final_imputed_data.precipitation_inches

3665-element SentinelArrays.SentinelArray{Float64,1,Float64,Missing,Array{Float64,1}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0